# U-Net

### 목표
이 노트북에서는 생물의학 이미징 분할화 작업을 위해 U-Net을 구현합니다. 특히, 뉴런에 레이블을 붙일 것이므로 이것을 신경 신경망(neural neural network)이라고 부를 수 있습니다! ;)

이것은 GAN, 생성 모델 또는 비지도학습도 아닙니다. 이것은 지도학습 이므로 정답은 하나뿐입니다(예: 분류기!) 이 구성 요소가 이번 주 다음 노트북에서 Pix2Pix의 Generator 구성 요소의 기초가 되는 방법을 볼 수 있습니다.

### 학습 목표
1. 자신만의 U-Net을 구현합니다.
2. 까다로운 분할(segmentation) 작업에서 U-Net의 성능을 관찰하십시오.

## 시작하기
먼저 라이브러리를 가져오고, 시각화 기능을 정의하고, 사용할 신경 데이터 세트를 가져옵니다.

#### 데이터세트
이 노트북의 경우 전자 현미경 데이터 세트를 사용합니다.
이미지 및 분할 데이터. 사용하게 될 데이터세트에 대한 정보는 [여기](https://www.ini.uzh.ch/~acardona/data.html)에서 확인하실 수 있습니다!

> Arganda-Carreras et al. "이미지 생성 크라우드소싱
Connectomics를 위한 세분화 알고리즘". Front. Neuroanat. 2015. https://www.frontiersin.org/articles/10.3389/fnana.2015.00142/full

![dataset example](Neuraldatasetexample.png)

In [1]:
from google.colab import drive
drive.mount('/content/drive')
# click the link and copy the code and paste it into the box below !

Mounted at /content/drive


In [2]:
cd drive/MyDrive/Classes/GAN/C3W2/Exercises/ExercisesA/

/content/drive/MyDrive/Classes/GAN/C3W2/Exercises/ExercisesA


In [3]:
pwd

'/content/drive/MyDrive/Classes/GAN/C3W2/Exercises/ExercisesA'

In [ ]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0)

def show_tensor_images(image_tensor, num_images=25, size=(1, 28, 28)):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in an uniform grid.
    '''
    # image_shifted = (image_tensor + 1) / 2
    image_shifted = image_tensor
    image_unflat = image_shifted.detach().cpu().view(-1, *size)
    image_grid = make_grid(image_unflat[:num_images], nrow=4)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

## U-Net 아키텍처
이제 구성 요소들로부터 U-Net을 구축할 수 있습니다. 아래 그림은 Ronneberger 등의 2015 논문 [*U-Net: Convolutional Networks for Biomedical Image Segmentation*](https://arxiv.org/abs/1505.04597)에서 가져온 것입니다. 그것은 U-Net 아키텍처와 그것이 어떻게 축소되고 확장되는지 보여줍니다.

<!-- "[i]t consists of a contracting path (left side) and an expansive path (right side)" (Renneberger, 2015) -->

![Figure 1 from the paper, U-Net: Convolutional Networks for Biomedical Image Segmentation](https://drive.google.com/uc?export=view&id=1XgJRexE2CmsetRYyTLA7L8dsEwx7aQZY)

즉, 이미지는 먼저 높이와 너비를 줄이는 동시에 채널을 늘리는 많은 컨볼루션 레이어를 통해 공급됩니다. 이 레이어를 저자는 "수축 경로"라고 부릅니다. 예를 들어, 보폭이 2인 2개의 2 x 2 컨볼루션 세트는 1 x 28 x 28(채널, 높이, 너비) 회색조 이미지를 취하여 2 x 14 x 14 표현을 만들게 됩니다. "확장 경로"는 이와 반대로 점점 더 적은 수의 채널로 이미지를 점차적으로 성장시킵니다.

## 수축 경로(contracting path)
먼저 수축 경로에 대한 수축 블록을 구현합니다. 이 경로는 U-Net의 인코더 섹션으로, 그 일부로 여러 다운샘플링 단계가 있습니다. 저자는 논문의 다음 단락에서 나머지 부분에 대해 자세히 설명합니다(Renneberger, 2015).

>축소 경로는 일반적인 컨볼루션 네트워크 아키텍처를 따릅니다. 이것은 2개의 3 x 3 컨볼루션(패딩되지 않은 컨볼루션)의 반복적인 적용으로 구성되며, 각각은 ReLU 와 다운샘플링을 위해 보폭 2를 사용하는 2 x 2 최대 풀링 작업이 뒤따릅니다. 각 다운샘플링 단계에서 특징 채널 수를 두 배로 늘립니다.

<details>
<summary>
<font size="3" color="green">
<b>Optional hints for <code><font size="4">ContractingBlock</font></code></b>
</font>
</summary>

1. 두 컨볼루션 모두 3 x 3 커널을 사용해야 합니다.  
2. 최대 풀링은 보폭이 2인 2 x 2 커널을 사용해야 합니다.
</details>

In [ ]:
# UNQ_C1 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED CLASS: ContractingBlock
class ContractingBlock(nn.Module):
    '''
    ContractingBlock Class
    Performs two convolutions followed by a max pool operation.
    Values:
        input_channels: the number of channels to expect from a given input
    '''
    def __init__(self, input_channels):
        super(ContractingBlock, self).__init__()
        # You want to double the number of channels in the first convolution
        # and keep the same number of channels in the second.
        #### START CODE HERE (~4 lines)####

        
        
        
        #### END CODE HERE ####

    def forward(self, x):
        '''
        Function for completing a forward pass of ContractingBlock: 
        Given an image tensor, completes a contracting block and returns the transformed tensor.
        Parameters:
            x: image tensor of shape (batch size, channels, height, width)
        '''
        x = self.conv1(x)
        x = self.activation(x)
        x = self.conv2(x)
        x = self.activation(x)
        x = self.maxpool(x)
        return x
    
    # Required for grading
    def get_self(self):
        return self

In [ ]:
#UNIT TEST
def test_contracting_block(test_samples=100, test_channels=10, test_size=50):
    test_block = ContractingBlock(test_channels)
    test_in = torch.randn(test_samples, test_channels, test_size, test_size)
    test_out_conv1 = test_block.conv1(test_in)
    # Make sure that the first convolution has the right shape
    assert tuple(test_out_conv1.shape) == (test_samples, test_channels * 2, test_size - 2, test_size - 2)
    # Make sure that the right activation is used
    assert torch.all(test_block.activation(test_out_conv1) >= 0)
    assert torch.max(test_block.activation(test_out_conv1)) >= 1
    test_out_conv2 = test_block.conv2(test_out_conv1)
    # Make sure that the second convolution has the right shape
    assert tuple(test_out_conv2.shape) == (test_samples, test_channels * 2, test_size - 4, test_size - 4)
    test_out = test_block(test_in)
    # Make sure that the pooling has the right shape
    assert tuple(test_out.shape) == (test_samples, test_channels * 2, test_size // 2 - 2, test_size // 2 - 2)

test_contracting_block()
test_contracting_block(10, 9, 8)
print("Success!")

## 확장 경로
다음으로 확장 경로에 대한 확장 블록을 구현합니다. 이것은 부분적으로 몇개의 업샘플링 단계가 있는 U-Net의 디코딩 섹션입니다. 이렇게 하려면 자르기 함수도 작성해야 합니다. 이는 *축소 경로*에서 이미지를 자르고 확장 경로의 현재 이미지에 연결할 수 있도록 하기 위한 것입니다. 이는 건너뛰기 연결을 형성하기 위한 것입니다. 다시 말하지만, 세부 사항은 논문에서 가져온 것입니다(Renneberger, 2015):

>확장 경로의 모든 단계는 피쳐 맵의 업샘플링과 피쳐 채널 수를 절반으로 줄이는 2 x 2 컨볼루션("업컨볼루션"), 축소 경로에서 해당하는 잘린 피쳐 맵과의 연결로 구성됩니다. 2개의 3 x 3 컨볼루션과 각각 ReLU가 뒤따릅니다. 모든 컨볼루션에서 경계 픽셀이 손실되기 때문에 자르기가 필요합니다.

<!-- so that the expanding block can resize the input from the contracting block can have the same size as the input from the previous layer -->

*Fun fact: 이 아키텍처를 기반으로 하는 이후 모델은 종종 컨볼루션에서 패딩을 사용하여 이미지 크기가 업샘플링/다운샘플링 단계 외부에서 변경되는 것을 방지합니다!*

<details>
<summary>
<font size="3" color="green">
<b>Optional hint for <code><font size="4">ExpandingBlock</font></code></b>
</font>
</summary>

1. 연결은 채널 수가 다시 input_channels로 돌아가는 것을 의미하므로 다음 convolution을 위해 다시 반으로 줄여야 합니다.
</details>

In [ ]:
# UNQ_C2 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: crop
def crop(image, new_shape):
    '''
    Function for cropping an image tensor: Given an image tensor and the new shape,
    crops to the center pixels.
    Parameters:
        image: image tensor of shape (batch size, channels, height, width)
        new_shape: a torch.Size object with the shape you want x to have
    '''
    # There are many ways to implement this crop function, but it's what allows
    # the skip connection to function as intended with two differently sized images!
    #### START CODE HERE (~7lines)####

    
    
    
    
    
    
    #### END CODE HERE ####
    return cropped_image

In [ ]:
#UNIT TEST
def test_expanding_block_crop(test_samples=100, test_channels=10, test_size=100):
    # Make sure that the crop function is the right shape
    skip_con_x = torch.randn(test_samples, test_channels, test_size + 6, test_size + 6)
    x = torch.randn(test_samples, test_channels, test_size, test_size)
    cropped = crop(skip_con_x, x.shape)
    assert tuple(cropped.shape) == (test_samples, test_channels, test_size, test_size)

    # Make sure that the crop function takes the right area
    test_meshgrid = torch.meshgrid([torch.arange(0, test_size), torch.arange(0, test_size)])
    test_meshgrid = test_meshgrid[0] + test_meshgrid[1]
    test_meshgrid = test_meshgrid[None, None, :, :].float()
    cropped = crop(test_meshgrid, torch.Size([1, 1, test_size // 2, test_size // 2]))
    assert cropped.max() == (test_size - 1) * 2 - test_size // 2
    assert cropped.min() == test_size // 2
    assert cropped.mean() == test_size - 1

    test_meshgrid = torch.meshgrid([torch.arange(0, test_size), torch.arange(0, test_size)])
    test_meshgrid = test_meshgrid[0] + test_meshgrid[1]
    crop_size = 5
    test_meshgrid = test_meshgrid[None, None, :, :].float()
    cropped = crop(test_meshgrid, torch.Size([1, 1, crop_size, crop_size]))
    assert cropped.max() <= (test_size + crop_size - 1) and cropped.max() >= test_size - 1
    assert cropped.min() >= (test_size - crop_size - 1) and cropped.min() <= test_size - 1
    assert abs(cropped.mean() - test_size) <= 2

test_expanding_block_crop()
print("Success!")

In [ ]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED CLASS: ExpandingBlock
class ExpandingBlock(nn.Module):
    '''
    ExpandingBlock Class
    Performs an upsampling, a convolution, a concatenation of its two inputs,
    followed by two more convolutions.
    Values:
        input_channels: the number of channels to expect from a given input
    '''
    def __init__(self, input_channels):
        super(ExpandingBlock, self).__init__()
        # "Every step in the expanding path consists of an upsampling of the feature map"
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        # "followed by a 2x2 convolution that halves the number of feature channels"
        # "a concatenation with the correspondingly cropped feature map from the contracting path"
        # "and two 3x3 convolutions"
        #### START CODE HERE (~3 lines)####

        
        
        #### END CODE HERE ####
        self.activation = nn.ReLU() # "each followed by a ReLU"
 
    def forward(self, x, skip_con_x):
        '''
        Function for completing a forward pass of ExpandingBlock: 
        Given an image tensor, completes an expanding block and returns the transformed tensor.
        Parameters:
            x: image tensor of shape (batch size, channels, height, width)
            skip_con_x: the image tensor from the contracting path (from the opposing block of x)
                    for the skip connection
        '''
        x = self.upsample(x)
        x = self.conv1(x)
        skip_con_x = crop(skip_con_x, x.shape)
        x = torch.cat([x, skip_con_x], axis=1)
        x = self.conv2(x)
        x = self.activation(x)
        x = self.conv3(x)
        x = self.activation(x)
        return x
    
    # Required for grading
    def get_self(self):
        return self

In [ ]:
#UNIT TEST
def test_expanding_block(test_samples=100, test_channels=10, test_size=50):
    test_block = ExpandingBlock(test_channels)
    skip_con_x = torch.randn(test_samples, test_channels // 2, test_size * 2 + 6, test_size * 2 + 6)
    x = torch.randn(test_samples, test_channels, test_size, test_size)
    x = test_block.upsample(x)
    x = test_block.conv1(x)
    # Make sure that the first convolution produces the right shape
    assert tuple(x.shape) == (test_samples, test_channels // 2,  test_size * 2 - 1, test_size * 2 - 1)
    orginal_x = crop(skip_con_x, x.shape)
    x = torch.cat([x, orginal_x], axis=1)
    x = test_block.conv2(x)
    # Make sure that the second convolution produces the right shape
    assert tuple(x.shape) == (test_samples, test_channels // 2,  test_size * 2 - 3, test_size * 2 - 3)
    x = test_block.conv3(x)
    # Make sure that the final convolution produces the right shape
    assert tuple(x.shape) == (test_samples, test_channels // 2,  test_size * 2 - 5, test_size * 2 - 5)
    x = test_block.activation(x)

test_expanding_block()
print("Success!")

## 최종 레이어
이제 최종 특징 매핑 블록을 작성합니다. 이 블록은 임의의 많은 텐서가 있는 텐서를 받아 픽셀 수는 같지만 정확한 수의 출력 채널을 가진 텐서를 생성합니다. 논문(Renneberger, 2015) 에서:

>최종 레이어에서 1x1 컨볼루션을 사용하여 각 64개 구성 요소 특징 벡터를 원하는 클래스 수에 매핑합니다. 네트워크에는 총 23개의 컨볼루션 레이어가 있습니다.


In [ ]:
# UNQ_C4 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED CLASS: FeatureMapBlock
class FeatureMapBlock(nn.Module):
    '''
    FeatureMapBlock Class
    The final layer of a UNet - 
    maps each pixel to a pixel with the correct number of output dimensions
    using a 1x1 convolution.
    Values:
        input_channels: the number of channels to expect from a given input
    '''
    def __init__(self, input_channels, output_channels):
        super(FeatureMapBlock, self).__init__()
        # "Every step in the expanding path consists of an upsampling of the feature map"
        #### START CODE HERE (~1 line)####

        #### END CODE HERE ####

    def forward(self, x):
        '''
        Function for completing a forward pass of FeatureMapBlock: 
        Given an image tensor, returns it mapped to the desired number of channels.
        Parameters:
            x: image tensor of shape (batch size, channels, height, width)
        '''
        x = self.conv(x)
        return x

In [ ]:
# UNIT TEST
assert tuple(FeatureMapBlock(10, 60)(torch.randn(1, 10, 10, 10)).shape) == (1, 60, 10, 10)
print("Success!")

## U-Net

이제 모두 함께 합칠 수 있습니다! 여기에서 구현한 세 가지 종류의 블록을 결합하는 `UNet` 클래스를 작성합니다.

In [ ]:
# UNQ_C5 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED CLASS: UNet
class UNet(nn.Module):
    '''
    UNet Class
    A series of 4 contracting blocks followed by 4 expanding blocks to 
    transform an input image into the corresponding paired image, with an upfeature
    layer at the start and a downfeature layer at the end
    Values:
        input_channels: the number of channels to expect from a given input
        output_channels: the number of channels to expect for a given output
    '''
    def __init__(self, input_channels, output_channels, hidden_channels=64):
        super(UNet, self).__init__()
        # "Every step in the expanding path consists of an upsampling of the feature map"
        self.upfeature = FeatureMapBlock(input_channels, hidden_channels)
        self.contract1 = ContractingBlock(hidden_channels)
        self.contract2 = ContractingBlock(hidden_channels * 2)
        self.contract3 = ContractingBlock(hidden_channels * 4)
        self.contract4 = ContractingBlock(hidden_channels * 8)
        self.expand1 = ExpandingBlock(hidden_channels * 16)
        self.expand2 = ExpandingBlock(hidden_channels * 8)
        self.expand3 = ExpandingBlock(hidden_channels * 4)
        self.expand4 = ExpandingBlock(hidden_channels * 2)
        self.downfeature = FeatureMapBlock(hidden_channels, output_channels)

    def forward(self, x):
        '''
        Function for completing a forward pass of UNet: 
        Given an image tensor, passes it through U-Net and returns the output.
        Parameters:
            x: image tensor of shape (batch size, channels, height, width)
        '''
        # Keep in mind that the expand function takes two inputs, 
        # both with the same number of channels. 
        #### START CODE HERE (~10 lines)####

        
        
        
        
        
        
        
        
        
        
        #### END CODE HERE ####
        return xn

In [ ]:
#UNIT TEST
test_unet = UNet(1, 3)
assert tuple(test_unet(torch.randn(1, 1, 256, 256)).shape) == (1, 3, 117, 117)
print("Success!")

## 훈련

마침내, 당신은 이것을 행동으로 옮기게 될 것입니다!
매개변수는 다음과 같습니다.
   * criterion: 손실 함수
   * n_epochs: 훈련 시 전체 데이터 세트를 반복하는 횟수
   * input_dim: 입력 이미지의 채널 수
   * label_dim: 출력 이미지의 채널 수
   * display_step: 이미지를 표시/시각화하는 빈도
   * batch_size: 정방향/역방향 패스당 이미지 수
   * lr: 학습률
   * initial_shape: 입력 이미지의 크기(픽셀 단위)
   * target_shape: 출력 이미지의 크기(픽셀 단위)
   * device: 장치 유형

훈련하는 데 몇 분 밖에 걸리지 않습니다!


In [ ]:
import torch.nn.functional as F
criterion = nn.BCEWithLogitsLoss()
n_epochs = 100
input_dim = 1
label_dim = 1
display_step = 100
batch_size = 4
lr = 0.0002
initial_shape = 512
target_shape = 373
device = 'cuda'
#device = 'cpu'

In [ ]:
from skimage import io
import numpy as np

volumes = torch.Tensor(io.imread('train-volume.tif'))[:, None, :, :] / 255
labels = torch.Tensor(io.imread('train-labels.tif', plugin="tifffile"))[:, None, :, :] / 255
labels = crop(labels, torch.Size([len(labels), 1, target_shape, target_shape]))
dataset = torch.utils.data.TensorDataset(volumes, labels)

In [ ]:
def train():
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True)
    unet = UNet(input_dim, label_dim).to(device)
    unet_opt = torch.optim.Adam(unet.parameters(), lr=lr)
    cur_step = 0

    for epoch in range(n_epochs):
        for real, labels in tqdm(dataloader):
            cur_batch_size = len(real)
            # Flatten the image
            real = real.to(device)
            labels = labels.to(device)

            ### Update U-Net ###
            unet_opt.zero_grad()
            pred = unet(real)
            unet_loss = criterion(pred, labels)
            unet_loss.backward()
            unet_opt.step()

            if cur_step % display_step == 0 or display_step == 800:
                print(f"Epoch {epoch}: Step {cur_step}: U-Net loss: {unet_loss.item()}")
                show_tensor_images(
                    crop(real, torch.Size([len(real), 1, target_shape, target_shape])), 
                    size=(input_dim, target_shape, target_shape)
                )
                show_tensor_images(labels, size=(label_dim, target_shape, target_shape))
                show_tensor_images(torch.sigmoid(pred), size=(label_dim, target_shape, target_shape))
            cur_step += 1

train()